In [ ]:
import zipfile
import os

zip_file = "/mnt/sda1/shuof/HW_LLM/HW2/HW2_Introduction/data.zip"

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall("/mnt/sda1/shuof/HW_LLM/HW2/HW2_Introduction")

In [24]:
!pip3 install -U packaging==23.2 setuptools==75.8.0 wheel ninja
!pip3 install --no-build-isolation axolotl[flash-attn,deepspeed]

# Download example axolotl configs, deepspeed configs
#axolotl fetch examples
#axolotl fetch deepspeed_configs  # OPTIONAL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 11.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp310-cp310-linux_x86_64.whl size=189909262 sha256=52226aec6dcb80b89ccb16e724a42eaaa90ecc08726e7fcd0ec0db38352c4744
  Stored in directory: /home/shuof/.cache/pip/wheels/59/ce/d5/08ea07bfc16ba218dc65a3a7ef9b6a270530bcbd2cea2ee1ca
Successfully built flash-attn


Training data format transform. [link](https://axolotl-ai-cloud.github.io/axolotl/docs/dataset-formats/)

In [ ]:
import json

input_file = "/mnt/sda1/shuof/HW_LLM/HW2/HW2_Introduction/data/train.json"  # paper_id, abstract, introduction
output_file = "axolotl_format_llama.jsonl"

# instruction
instruction_text = "You are a professional researcher writing for a top-tier academic conference. Based on the introduction, write a concise and formal abstract that clearly presents the motivation, methodology, and contributions of the work.\n"

with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
    for line in infile:
        data = json.loads(line)
        introduction = data.get("introduction", "").strip()
        abstract = data.get("abstract", "").strip()

        if introduction and abstract:
            new_data = {
                "instruction": "{instruction_text}\n\n",
                "input": f"{introduction}\n",
                "output": abstract
            }
            json.dump(new_data, outfile)
            outfile.write("\n")

print(f"✅ 已轉換完成，結果儲存在 {output_file}")

✅ 已轉換完成，結果儲存在 axolotl_format_llama.jsonl


In [ ]:
import json

input_file = "/mnt/sda1/shuof/HW_LLM/HW2/HW2_Introduction/data/test.json"  # paper_id, introduction
output_file = "test_axolotl_format_llama.json"

# instruction
instruction_text = "You are a professional researcher writing for a top-tier academic conference. Based on the introduction, write a concise and formal abstract that clearly presents the motivation, methodology, and contributions of the work.\n"

with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
    for line in infile:
        data = json.loads(line)
        introduction = data.get("introduction", "").strip()

        if introduction:
            new_data = {
                "instruction": "{instruction_text}\n\n",
                "input": f"{introduction}\n",
            }
            json.dump(new_data, outfile)
            outfile.write("\n")

print(f"✅ 已轉換完成，結果儲存在 {output_file}")

✅ 已轉換完成，結果儲存在 test_axolotl_format_llama.json


## train

In [1]:
import subprocess

# train model
subprocess.run(["axolotl", "train", "qlora-fsdp.yaml"])

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.


[2025-04-08 09:56:16,630] [INFO] [datasets.<module>:54] [PID:770857] PyTorch version 2.6.0+cu118 available.
[2025-04-08 09:56:16,630] [INFO] [datasets.<module>:54] [PID:770856] PyTorch version 2.6.0+cu118 available.
[2025-04-08 09:56:17,209] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-04-08 09:56:17,222] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-04-08 09:56:17,258] [INFO] [root.spawn:60] [PID:770856] gcc -pthread -B /home/shuof/.conda/envs/llm2/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/shuof/.conda/envs/llm2/include -fPIC -O2 -isystem /home/shuof/.conda/envs/llm2/include -fPIC -c /tmp/tmpe8_5y599/test.c -o /tmp/tmpe8_5y599/test.o
[2025-04-08 09:56:17,269] [INFO] [root.spawn:60] [PID:770856] gcc -pthread -B /home/shuof/.conda/envs/llm2/compiler_compat /tmp/tmpe8_5y599/test.o -laio -o /tmp/tmpe8_5y599/a.out
[2025-04-

[rank1]:[W408 09:56:19.123387369 ProcessGroupNCCL.cpp:4561] [PG ID 0 PG GUID 0 Rank 1]  using GPU 1 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. Specify device_ids in barrier() to force use of a particular device, or call init_process_group() with a device_id.


[2025-04-08 09:56:19,735] [INFO] [axolotl.utils.data.sft.get_dataset_wrapper:457] [PID:770856] [RANK:0] Loading dataset with base_type: alpaca and prompt_style: None
[2025-04-08 09:56:19,824] [DEBUG] [axolotl.utils.data.utils.drop_long_seq_in_dataset:176] [PID:770856] [RANK:0] min_input_len: 341
[2025-04-08 09:56:19,825] [DEBUG] [axolotl.utils.data.utils.drop_long_seq_in_dataset:178] [PID:770856] [RANK:0] max_input_len: 2478
[2025-04-08 09:56:20,126] [WARNING] [axolotl.utils.data.utils.drop_long_seq_in_dataset:206] [PID:770856] [RANK:0] Dropped 34 long samples from dataset
[2025-04-08 09:56:20,500] [INFO] [axolotl.utils.data.sft.load_tokenized_prepared_datasets:332] [PID:770856] [RANK:0] Saving merged prepared dataset to disk... last_run_prepared/5606787b0f411587f91aaeb73be5d281


Saving the dataset (1/1 shards): 100%|██████████| 374/374 [00:00<00:00, 5372.34 examples/s]
[rank0]:[W408 09:56:20.638384669 ProcessGroupNCCL.cpp:4561] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. Specify device_ids in barrier() to force use of a particular device, or call init_process_group() with a device_id.


...

{'loss': 0.6465, 'grad_norm': 1.4981250762939453, 'learning_rate': 0.0, 'epoch': 3.92}
[2025-04-08 10:48:18,649] [INFO] [accelerate.utils.fsdp_utils.save_fsdp_model:89] [PID:770856] Saving model to ./model-out/checkpoint-176/pytorch_model_fsdp.bin
[2025-04-08 10:48:22,085] [INFO] [accelerate.utils.fsdp_utils.save_fsdp_model:91] [PID:770856] Model saved to ./model-out/checkpoint-176/pytorch_model_fsdp.bin


/home/shuof/.conda/envs/llm2/lib/python3.10/site-packages/torch/distributed/distributed_c10d.py:863: UserWarning: `_get_pg_default_device` will be deprecated, it only stays for backward-compatiblity reason. If you need to find a device for object collectives, please use `_get_object_coll_device`. If you need to query the device types supported by group, please use `_device_capability(group)`. 
  warnings.warn(


[2025-04-08 10:48:22,670] [INFO] [accelerate.utils.fsdp_utils.save_fsdp_optimizer:193] [PID:770856] Saving Optimizer state to ./model-out/checkpoint-176/optimizer.bin
[2025-04-08 10:48:29,984] [INFO] [accelerate.utils.fsdp_utils.save_fsdp_optimizer:195] [PID:770856] Optimizer state saved in ./model-out/checkpoint-176/optimizer.bin
{'train_runtime': 3113.0634, 'train_samples_per_second': 0.481, 'train_steps_per_second': 0.057, 'train_loss': 1.0278361282226713, 'epoch': 3.92}
[2025-04-08 10:48:30,103] [INFO] [axolotl.train.train:192] [PID:770856] [RANK:0] Training Completed!!! Saving pre-trained model to ./model-out
[2025-04-08 10:48:30,112] [INFO] [axolotl.train.train:204] [PID:770856] [RANK:0] Set FSDP state dict type to FULL_STATE_DICT for saving.


100%|██████████| 176/176 [51:53<00:00, 17.69s/it]
/home/shuof/.conda/envs/llm2/lib/python3.10/site-packages/torch/distributed/fsdp/fully_sharded_data_parallel.py:690: FutureWarning: FSDP.state_dict_type() and FSDP.set_state_dict_type() are being deprecated. Please use APIs, get_state_dict() and set_state_dict(), which can support different parallelisms, FSDP1, FSDP2, DDP. API doc: https://pytorch.org/docs/stable/distributed.checkpoint.html#torch.distributed.checkpoint.state_dict.get_state_dict .Tutorial: https://pytorch.org/tutorials/recipes/distributed_checkpoint_recipe.html .
  warnings.warn(
[rank0]:[W408 10:48:41.427339333 ProcessGroupNCCL.cpp:1496] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


CompletedProcess(args=['axolotl', 'train', 'qlora-fsdp.yaml'], returncode=0)

In [3]:
# another training method (server 209 has only one GPU -> OOM)
"""
import yaml
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset

with open("qlora-fsdp.yaml", "r") as f:
    config = yaml.safe_load(f)

base_model = config.get("base_model", "Qwen/Qwen2-7B")
lora_cfg = config.get("lora_config", {})
train_cfg = config.get("training", {})
dataset_cfg = config.get("datasets", [{}])[0]

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": torch.cuda.current_device()},
    trust_remote_code=True
)
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=lora_cfg.get("r", 32),
    lora_alpha=lora_cfg.get("lora_alpha", 64),
    lora_dropout=lora_cfg.get("lora_dropout", 0.05),
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.gradient_checkpointing_enable() 

data_path = dataset_cfg.get("path", "./train.jsonl")
dataset = load_dataset("json", data_files=data_path)["train"]

def formatting_func(example):
    prompt = f"{example['instruction']}\n\n{example['input']}"
    
    inputs = tokenizer(prompt, truncation=True, max_length=2048)
    outputs = tokenizer(example["output"], truncation=True, max_length=700)

    input_ids = inputs["input_ids"]
    label_ids = outputs["input_ids"]

    return {
        "input_ids": input_ids,
        "attention_mask": inputs["attention_mask"],
        "labels": label_ids
    }

dataset = dataset.map(formatting_func, remove_columns=dataset.column_names)

training_args = TrainingArguments(
    output_dir="./qlora-output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=4,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    bf16=True,
    save_total_limit=1,
    report_to="none",

    # FSDP
    fsdp="full_shard auto_wrap",
    fsdp_transformer_layer_cls_to_wrap="Qwen2DecoderLayer",
    fsdp_config={
        "fsdp_limit_all_gathers": True,
        "fsdp_sync_module_states": True,
        "fsdp_offload_params": True,
        "fsdp_use_orig_params": False,
        "fsdp_cpu_ram_efficient_loading": True,
        "fsdp_auto_wrap_policy": "TRANSFORMER_BASED_WRAP",
        "fsdp_state_dict_type": "FULL_STATE_DICT",
        "fsdp_sharding_strategy": "FULL_SHARD"
    }
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="pt")

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

trainer.train()

model.save_pretrained("./qlora-output2/final")
tokenizer.save_pretrained("./qlora-output2/final")
"""

'\nimport yaml\nimport torch\nfrom transformers import (\n    AutoTokenizer,\n    AutoModelForCausalLM,\n    BitsAndBytesConfig,\n    TrainingArguments,\n    Trainer,\n    DataCollatorForSeq2Seq\n)\nfrom peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training\nfrom datasets import load_dataset\n\nwith open("qlora-fsdp.yaml", "r") as f:\n    config = yaml.safe_load(f)\n\nbase_model = config.get("base_model", "Qwen/Qwen2-7B")\nlora_cfg = config.get("lora_config", {})\ntrain_cfg = config.get("training", {})\ndataset_cfg = config.get("datasets", [{}])[0]\n\nbnb_config = BitsAndBytesConfig(\n    load_in_4bit=True,\n    bnb_4bit_compute_dtype=torch.bfloat16,\n    bnb_4bit_use_double_quant=True,\n    bnb_4bit_quant_type="nf4"\n)\n\ntokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)\nmodel = AutoModelForCausalLM.from_pretrained(\n    base_model,\n    quantization_config=bnb_config,\n    device_map={"": torch.cuda.current_device()},\n    trust_remote_c

## Predict

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
import json
from tqdm import tqdm

# path
base_model_path = "Qwen/Qwen2-7B"
adapter_path = "/mnt/sda1/shuof/HW_LLM/HW2/model-out"
input_file = "/mnt/sda1/shuof/HW_LLM/HW2/HW2_Introduction/data/test_axolotl_format_llama.json"
output_file = "output_QWEN2_2.json"

# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()

# Instruction prompt
instruction = (
    "You are a professional researcher writing for a top-tier academic conference. "
    "Based on the introduction, write a concise and formal abstract that clearly presents "
    "the motivation, methodology, and contributions of the work."
)

# tqdm
with open(input_file, "r", encoding="utf-8") as infile:
    lines = infile.readlines()

with open(output_file, "w", encoding="utf-8") as outfile:
    for line in tqdm(lines, desc="🔍 Generating Abstracts"):
        try:
            data = json.loads(line)
        except json.JSONDecodeError:
            continue

        paper_id = str(data.get("paper_id", "")).strip()
        intro = data.get("introduction", "").strip()

        prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{intro}\n\n### Response:\n"

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=1024,
                do_sample=False,
                eos_token_id=tokenizer.eos_token_id
            )
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        response = decoded.split("### Response:")[-1].strip()

        json.dump({
            "paper_id": paper_id,
            "abstract": response
        }, outfile, ensure_ascii=False)
        outfile.write("\n")

print(f"✅ 推論完成，已輸出至：{output_file}")

/home/shuof/.conda/envs/llm2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
🔍 Generating Abstracts: 100%|██████████| 103/103 [18:52<00:00, 10.99s/it]

✅ 推論完成，已輸出至：output_QWEN2_2.json


## Evaluate

In [ ]:
import json

with open("/mnt/sda1/shuof/HW_LLM/HW2/HW2_Introduction/data/train.json", "r") as f:
    train_data = [json.loads(line) for line in f]
    train_data = train_data[:103]

refs = [entry["abstract"] for entry in train_data]

with open("/mnt/sda1/shuof/HW_LLM/HW2/output_QWEN2.json", "r") as f:
    test_data = [json.loads(line) for line in f]
    test_data = test_data

preds = [entry["abstract"] for entry in test_data]

In [8]:
import evaluate

metric_rouge = evaluate.load("rouge", rouge_types=["rouge1", "rouge2", "rougeL"])
metric_bertscore = evaluate.load ("bertscore")
rouge = metric_rouge.compute (predictions=preds, references=refs, use_stemmer=True)
bertscore = metric_bertscore.compute(predictions=preds, references=refs, lang="en")

print(rouge)
print(bertscore)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'rouge1': np.float64(0.3630904045105764), 'rouge2': np.float64(0.06626940457958805), 'rougeL': np.float64(0.1797194956005157), 'rougeLsum': np.float64(0.17948802720259488)}
{'precision': [0.8332086801528931, 0.8455277681350708, 0.8418658971786499, 0.8484259247779846, 0.8263354301452637, 0.8333075046539307, 0.8365870118141174, 0.8522847294807434, 0.8509970903396606, 0.8371009230613708, 0.8677625060081482, 0.843564510345459, 0.8210716843605042, 0.8473855257034302, 0.832959771156311, 0.8347581624984741, 0.8392698764801025, 0.8446987271308899, 0.8558298349380493, 0.8426832556724548, 0.8440885543823242, 0.8410561084747314, 0.8371365070343018, 0.8374246954917908, 0.8246350288391113, 0.844743013381958, 0.8330234289169312, 0.8459643125534058, 0.8334879875183105, 0.8387441039085388, 0.8435492515563965, 0.8509434461593628, 0.8429733514785767, 0.8419063091278076, 0.847912073135376, 0.8334373235702515, 0.8438273668289185, 0.8451995849609375, 0.8429703712463379, 0.8497307300567627, 0.8419461250305